# **Population ETL**

***

## **Overview:**

This ETL will be focused on obtaining more information and knowledge about the population of Girona for future statistical studies


In [1]:
import pandas as pd
import requests
import datetime

## **Population data**

### **Extraction**

In [2]:
basic_headers = {'User-Agent': 'Mozilla/5.0'}
endpoint = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM'
payload = requests.get(endpoint, headers=basic_headers)

In [3]:
population_data = payload.json()

Extracting and cleaning data from a JSON structure, handling potential errors gracefully.

In [4]:
population_clean = []
pop_err = []
for reg in population_data:
    
    try:
        if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
            temp = {"id": reg['COD']}
            temp['Poblacion'] = reg['MetaData'][1]['Nombre']
            temp['Origen'] = reg['MetaData'][2]['Nombre']

            for row in reg['Data']:
                temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

            population_clean.append(temp)
    
    except Exception as e:
            pop_err.append(reg)
            print(e)

In [5]:
pop_err

[]

In [6]:
len(population_clean)

26520

In [7]:
population_std = pd.DataFrame(population_clean, columns=population_clean[0].keys())
population_std.head()

,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5662094,Agullana,Total,903.0,885.0,863.0,831.0,841.0,831.0,841.0,...,858.0,839.0,840.0,812.0,806.0,753.0,756.0,759.0,746.0,723.0
1,PC5662093,Agullana,España,735.0,711.0,704.0,689.0,697.0,692.0,704.0,...,678.0,657.0,651.0,645.0,648.0,645.0,662.0,670.0,657.0,664.0
2,PC5662092,Agullana,Extranjero,168.0,174.0,159.0,142.0,144.0,139.0,137.0,...,180.0,182.0,189.0,167.0,158.0,108.0,94.0,89.0,89.0,59.0
3,PC5662063,Agullana,Europa (sin España),71.0,79.0,67.0,56.0,57.0,53.0,55.0,...,94.0,103.0,103.0,92.0,84.0,59.0,51.0,55.0,53.0,33.0
4,PC5662062,Agullana,UE28 sin España,NaN,NaN,57.0,50.0,54.0,49.0,51.0,...,89.0,97.0,96.0,87.0,79.0,56.0,48.0,49.0,46.0,28.0


In [8]:
duplicates = population_std['id'].duplicated()
duplicated_df = population_std[duplicates]
duplicated_df

,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003


In [9]:
population_df = pd.DataFrame(population_clean)

In [10]:
population_df.head()

,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5662094,Agullana,Total,903.0,885.0,863.0,831.0,841.0,831.0,841.0,...,858.0,839.0,840.0,812.0,806.0,753.0,756.0,759.0,746.0,723.0
1,PC5662093,Agullana,España,735.0,711.0,704.0,689.0,697.0,692.0,704.0,...,678.0,657.0,651.0,645.0,648.0,645.0,662.0,670.0,657.0,664.0
2,PC5662092,Agullana,Extranjero,168.0,174.0,159.0,142.0,144.0,139.0,137.0,...,180.0,182.0,189.0,167.0,158.0,108.0,94.0,89.0,89.0,59.0
3,PC5662063,Agullana,Europa (sin España),71.0,79.0,67.0,56.0,57.0,53.0,55.0,...,94.0,103.0,103.0,92.0,84.0,59.0,51.0,55.0,53.0,33.0
4,PC5662062,Agullana,UE28 sin España,NaN,NaN,57.0,50.0,54.0,49.0,51.0,...,89.0,97.0,96.0,87.0,79.0,56.0,48.0,49.0,46.0,28.0


In [11]:
population_df.columns

Index([       'id', 'Poblacion',    'Origen',        2022,        2021,
              2020,        2019,        2018,        2017,        2016,
              2015,        2014,        2013,        2012,        2011,
              2010,        2009,        2008,        2007,        2006,
              2005,        2004,        2003],
      dtype='object')

In [12]:
population_df.drop(columns=['id'],inplace=True)

### **Load**


We filter the DataFrame `population_df` to include only rows where the value in the 'Origen' column is either 'Total', 'España', or 'Extranjero'. This operation helps us focus on specific population data related to total population, Spanish nationals, and foreigners.


In [13]:
population_df = population_df[(population_df['Origen'] == 'Total') | (population_df['Origen'] == 'España') | (population_df['Origen'] == 'Extranjero')]

As we can see, we have the rows for every city repeated three times. We only want the first appearence of each row which is the sum of the other two.

In [14]:
population_df[population_df['Poblacion'] == 'Girona']

,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
2960,Girona,Total,102666.0,101932.0,103369.0,101852.0,100266.0,99013.0,98255.0,97586.0,...,97198.0,96722.0,96236.0,96188.0,94484.0,92186.0,89890.0,86672.0,83531.0,81220.0
2961,Girona,España,76298.0,76325.0,77293.0,77618.0,77722.0,77516.0,77365.0,77034.0,...,76109.0,75763.0,75184.0,75121.0,74797.0,74833.0,74870.0,74689.0,74131.0,73224.0
2962,Girona,Extranjero,26368.0,25607.0,26076.0,24234.0,22544.0,21497.0,20890.0,20552.0,...,21089.0,20959.0,21052.0,21067.0,19687.0,17353.0,15020.0,11983.0,9400.0,7996.0
11800,Girona,Total,49352.0,49050.0,49809.0,48931.0,48030.0,47379.0,47149.0,46761.0,...,46865.0,46675.0,46883.0,47161.0,46380.0,45029.0,43863.0,41998.0,40340.0,39266.0
11801,Girona,España,36578.0,36689.0,37134.0,37342.0,37380.0,37250.0,37204.0,36924.0,...,36418.0,36174.0,35962.0,35897.0,35808.0,35815.0,35840.0,35721.0,35510.0,35083.0
11802,Girona,Extranjero,12774.0,12361.0,12675.0,11589.0,10650.0,10129.0,9945.0,9837.0,...,10447.0,10501.0,10921.0,11264.0,10572.0,9214.0,8023.0,6277.0,4830.0,4183.0
20640,Girona,Total,53314.0,52882.0,53560.0,52921.0,52236.0,51634.0,51106.0,50825.0,...,50333.0,50047.0,49353.0,49027.0,48104.0,47157.0,46027.0,44674.0,43191.0,41954.0
20641,Girona,España,39720.0,39636.0,40159.0,40276.0,40342.0,40266.0,40161.0,40110.0,...,39691.0,39589.0,39222.0,39224.0,38989.0,39018.0,39030.0,38968.0,38621.0,38141.0
20642,Girona,Extranjero,13594.0,13246.0,13401.0,12645.0,11894.0,11368.0,10945.0,10715.0,...,10642.0,10458.0,10131.0,9803.0,9115.0,8139.0,6997.0,5706.0,4570.0,3813.0


If we sum the total population by year we get the official population of Spain for every year.

In [15]:
population_df.drop_duplicates(subset=['Poblacion', 'Origen'], keep='first', inplace=True)

In [16]:
population_df[population_df['Poblacion'] == 'Girona']

,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
2960,Girona,Total,102666.0,101932.0,103369.0,101852.0,100266.0,99013.0,98255.0,97586.0,...,97198.0,96722.0,96236.0,96188.0,94484.0,92186.0,89890.0,86672.0,83531.0,81220.0
2961,Girona,España,76298.0,76325.0,77293.0,77618.0,77722.0,77516.0,77365.0,77034.0,...,76109.0,75763.0,75184.0,75121.0,74797.0,74833.0,74870.0,74689.0,74131.0,73224.0
2962,Girona,Extranjero,26368.0,25607.0,26076.0,24234.0,22544.0,21497.0,20890.0,20552.0,...,21089.0,20959.0,21052.0,21067.0,19687.0,17353.0,15020.0,11983.0,9400.0,7996.0



 We save the filtered DataFrame `population_df` to a CSV file named 'Girona_Population.csv'. This allows us to export the cleaned and filtered population data for further analysis or sharing with others.


In [17]:
population_df.to_csv('Girona_Population.csv')

## **Government data**

### **Extraction**

We use Pandas to read a CSV file named 'datosgobes.csv' into a DataFrame called `df`. We specify the encoding parameter as "utf-8" to ensure proper handling of Unicode characters during the file reading process

https://datos.gob.es/es/catalogo

In [18]:
df = pd.read_csv('datosgobes.csv', encoding = "utf-8")

In [19]:
df['DISTRIBUCIONES'][0:3]

0    [TITLE_es]pdf[ACCESS_URL]https://www.aepd.es/r...
1    [ACCESS_URL]https://www.aepd.es/informes/[MEDI...
2    [ACCESS_URL]https://www.boe.es/buscar/legislac...
Name: DISTRIBUCIONES, dtype: object

We filter the DataFrame `df` to find rows where the 'DISTRIBUCIONES' column contains the specific URL 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM'. We use the `str.contains()` method to perform the search, specifying `na=False` to exclude missing values and `regex=False` to indicate that we are searching for a literal string rather than a regular expression. The filtered rows are stored in the variable `my_row`, and we display the result.


In [20]:
my_row = df[df['DISTRIBUCIONES'].str.contains('https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM', na=False, regex=False)]
my_row

,URL,IDENTIFICADOR,TÍTULO,DESCRIPCIÓN,TEMÁTICAS,ETIQUETAS,FECHA DE CREACIÓN,FECHA DE ÚLTIMA MODIFICACIÓN,FRECUENCIA DE ACTUALIZACIÓN,IDIOMAS,ÓRGANO PUBLICADOR,CONDICIONES DE USO,COBERTURA GEOGRÁFICA,COBERTURA TEMPORAL,VIGENCIA DEL RECURSO,RECURSOS RELACIONADOS,NORMATIVA,DISTRIBUCIONES
60779,https://datos.gob.es/catalogo/b62c72eb-b291-40...,urn:ine:es:TABLA:T3:65:33791,"[en]Population by sex, municipalities and coun...","[en]Table of INEBase Population by sex, munici...",Demografía,[en]Abroad//Africa//Algeria//America//Brazil//...,2012-01-13T12:00:00+0100,2022-01-17T12:00:00+0100,[TYPE]http://www.w3.org/2006/time#years[VALUE]1,es//en,Instituto Nacional de Estadística,https://www.ine.es/aviso_legal,Girona,2003-01-01T01:00:00+0100-2022-01-01T01:00:00+0100,NaN,https://www.ine.es/dyngs/IOE/es/operacion.htm?...,https://www.ine.es/dynt3/metadatos/es/Respuest...,[TITLE_en]CSV: Tab Separated[TITLE_es]CSV: sep...


In [21]:
pd.set_option('display.max_colwidth', None)
my_row['DISTRIBUCIONES']

60779    [TITLE_en]CSV: Tab Separated[TITLE_es]CSV: separado por tabuladores[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bd/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]Excel: XLS extension[TITLE_es]Excel: Extensión XLS[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/xls/33791.xls[MEDIA_TYPE]XLS//[TITLE_en]Html[TITLE_es]Html[ACCESS_URL]https://www.ine.es/jaxiT3/Tabla.htm?t=33791[MEDIA_TYPE]HTML//[TITLE_en]Pc-Axis[TITLE_es]Pc-Axis[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/px/33791.px[MEDIA_TYPE]PC-Axis//[TITLE_en]Plain Text: separated by comma[TITLE_es]Texto plano: separado por ,[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_c/33791.csv_c[MEDIA_TYPE]CSV//[TITLE_en]Plain Text: separated by tab[TITLE_es]Texto plano: separado por tabuladores[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]CSV: Separated by ;[TITLE_es]CSV: separado por ;[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bdsc/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]Json[TITLE_es]Json[ACCESS_URL

We reset the display option `max_colwidth` to its default value using `pd.reset_option('display.max_colwidth')`

In [22]:
pd.reset_option('display.max_colwidth')
df.iloc[52701]['TÍTULO']

'[en]Alumnos que abandonaron la ESO en el curso 2013-14 según sus habilidades TIC por sexo y CCAA de estudio (API identifier: /t13/p451/2019/p09/l0/12005.px)[es]Alumnos que abandonaron la ESO en el curso 2013-14 según sus habilidades TIC por sexo y CCAA de estudio (Identificador API: /t13/p451/2019/p09/l0/12005.px)'

Let us reduce the dataframe to the rows of interest:

In [23]:
filtered_df = df[df['TÍTULO'].str.contains("[en]Population by sex, municipalities and country of birth", case=False, regex=False)]
print(len(filtered_df))

53


In [30]:
# filtered_df.to_csv('Data_of_interest.csv', index=False)
# filtered_df.to_excel('Data_of_interest.xlsx', index=False)

 We iterate over the indices of the DataFrame `filtered_df`. If the index matches 52701, we extract the distribution information from the 'DISTRIBUCIONES' column. We split this information to extract the link for JSON resources, then we print and store the link in the variable `mylink`.


In [24]:
mylink = ''
for i in filtered_df.index:
    if i==52701:
        print(filtered_df.loc[i].to_dict()['DISTRIBUCIONES'])
        dist = filtered_df.loc[i].to_dict()['DISTRIBUCIONES'].split('[TITLE_en]')

        for resource in dist:

            if resource[:4] == 'Json':
                print()
                print(resource)
                link = resource.split('[ACCESS_URL]')[1].split('[MEDIA_TYPE]')[0]
                print()
                print('Link -->',link)
                mylink = link



Let us proceed with all the rows:

In [25]:
distribuciones = []
n = 0

for i in filtered_df.index:
    dist = filtered_df.loc[i].to_dict()['DISTRIBUCIONES'].split('[TITLE_en]')
    
    for resource in dist:
        if resource[:4] == 'Json':
            link = resource.split('[ACCESS_URL]')[1].split('[MEDIA_TYPE]')[0]
            if link == mylink:
                print('-- The following is my link:')

                print(f'Link position: {n} -->',link)
            n += 1

            distribuciones.append(link)


In [26]:
print(len(distribuciones))
my_distributions = distribuciones[0:3]
my_distributions

53


['https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33713?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33779?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33845?tip=AM']


This function `extract_data()` retrieves population data from a list of distribution links. It iterates through each link, retrieves the JSON data using `requests.get()`, and then processes the JSON data to extract relevant population information. The extracted data is stored in a list `population_all` and returned at the end of the function. Any exceptions encountered during processing are printed, and the loop continues with the next link.


In [27]:
def extract_data(my_distributions):
    basic_headers = {'User-Agent': 'Mozilla/5.0'}
    population_all = []
    n = 0
    for link in my_distributions:
        print('Number of links processed:', n)
        n += 1
        
        payload = requests.get(link, headers=basic_headers)
        payload_json = payload.json()
        
        for reg in payload_json:

            try:
                if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
                    temp = {"id": reg['COD']}

                    temp['Poblacion'] = reg['MetaData'][1]['Nombre']
                    temp['Origen'] = reg['MetaData'][2]['Nombre']

                    for row in reg['Data']:
                        temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

                    population_all.append(temp)
            except Exception as e:
                print(e)
                pass
    
    return population_all

We applied the function

In [28]:
population_all = extract_data(distribuciones)

Number of links processed: 0
Number of links processed: 1
Number of links processed: 2
Number of links processed: 3
Number of links processed: 4
Number of links processed: 5
Number of links processed: 6
Number of links processed: 7
Number of links processed: 8
Number of links processed: 9
Number of links processed: 10
Number of links processed: 11
Number of links processed: 12
Number of links processed: 13
Number of links processed: 14
Number of links processed: 15
Number of links processed: 16
Number of links processed: 17
Number of links processed: 18
Number of links processed: 19
Number of links processed: 20
Number of links processed: 21
string indices must be integers, not 'str'
Number of links processed: 22
Number of links processed: 23
Number of links processed: 24
Number of links processed: 25
Number of links processed: 26
Number of links processed: 27
Number of links processed: 28
Number of links processed: 29
Number of links processed: 30
Number of links processed: 31
Number 

In [29]:
population_std = pd.DataFrame(population_all, columns=population_all[0].keys())
print(len(population_std))
population_std

975270


,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5660916,Alaior,Total,9606.0,9477.0,9467.0,9065.0,9012.0,9097.0,8959.0,...,9610.0,9600.0,9399.0,9257.0,9133.0,8972.0,8933.0,8671.0,8308.0,8197.0
1,PC5660915,Alaior,España,7927.0,7884.0,7778.0,7683.0,7633.0,7623.0,7574.0,...,7612.0,7610.0,7542.0,7489.0,7443.0,7370.0,7328.0,7249.0,7204.0,7210.0
2,PC5660914,Alaior,Extranjero,1679.0,1593.0,1689.0,1382.0,1379.0,1474.0,1385.0,...,1998.0,1990.0,1857.0,1768.0,1690.0,1602.0,1605.0,1422.0,1104.0,987.0
3,PC5660885,Alaior,Europa (sin España),785.0,703.0,780.0,605.0,665.0,795.0,776.0,...,1352.0,1316.0,1268.0,1187.0,1079.0,1030.0,1025.0,883.0,715.0,659.0
4,PC5660884,Alaior,UE28 sin España,NaN,NaN,735.0,562.0,624.0,759.0,739.0,...,1311.0,1281.0,1231.0,1156.0,1053.0,1005.0,995.0,868.0,702.0,644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975265,PC4742389,Viver i Serrateix,China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
975266,PC4742388,Viver i Serrateix,Pakistán,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
975267,PC22759735,Viver i Serrateix,UE27_2020 sin España,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975268,PC4742381,Viver i Serrateix,Oceanía,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **Transformation**


We save the DataFrame `population_std` to a CSV file named 'population_raw.csv'. This allows us to export the structured population data, but still raw.

In [30]:
population_std.to_csv('population_raw.csv')

In [31]:
population_clean = population_std.dropna()

In [32]:
population_clean.columns

Index([       'id', 'Poblacion',    'Origen',        2022,        2021,
              2020,        2019,        2018,        2017,        2016,
              2015,        2014,        2013,        2012,        2011,
              2010,        2009,        2008,        2007,        2006,
              2005,        2004,        2003],
      dtype='object')


In this code, we filter the DataFrame `population_clean` to include only rows where the value in the 'Origen' column is either 'Total', 'España', or 'Extranjero'. Then, we drop the 'id' column from the filtered DataFrame using the `drop()` method with `inplace=True`. This operation helps us focus on specific population data related to total population, Spanish nationals, and foreigners while removing the unnecessary 'id' column.


In [33]:
population_clean = population_clean[(population_clean['Origen'] == 'Total' )|(population_clean['Origen'] == 'España' )|(population_clean['Origen'] == 'Extranjero' )]
population_clean.drop(columns=['id'],inplace=True)

In [34]:
population_clean

,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,Alaior,Total,9606.0,9477.0,9467.0,9065.0,9012.0,9097.0,8959.0,8997.0,...,9610.0,9600.0,9399.0,9257.0,9133.0,8972.0,8933.0,8671.0,8308.0,8197.0
1,Alaior,España,7927.0,7884.0,7778.0,7683.0,7633.0,7623.0,7574.0,7643.0,...,7612.0,7610.0,7542.0,7489.0,7443.0,7370.0,7328.0,7249.0,7204.0,7210.0
2,Alaior,Extranjero,1679.0,1593.0,1689.0,1382.0,1379.0,1474.0,1385.0,1354.0,...,1998.0,1990.0,1857.0,1768.0,1690.0,1602.0,1605.0,1422.0,1104.0,987.0
38,Alaró,Total,5811.0,5741.0,5617.0,5572.0,5494.0,5389.0,5282.0,5275.0,...,5508.0,5438.0,5401.0,5327.0,5178.0,5018.0,4869.0,4707.0,4607.0,4540.0
39,Alaró,España,4411.0,4393.0,4344.0,4326.0,4334.0,4330.0,4309.0,4322.0,...,4282.0,4246.0,4230.0,4208.0,4139.0,4113.0,4024.0,3907.0,3885.0,3817.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975191,Vilobí del Penedès,España,495.0,500.0,488.0,495.0,481.0,476.0,480.0,476.0,...,495.0,491.0,497.0,500.0,492.0,482.0,468.0,462.0,443.0,431.0
975192,Vilobí del Penedès,Extranjero,46.0,43.0,44.0,43.0,43.0,43.0,44.0,45.0,...,48.0,43.0,46.0,38.0,38.0,32.0,30.0,31.0,27.0,17.0
975230,Viver i Serrateix,Total,81.0,87.0,84.0,81.0,78.0,85.0,76.0,78.0,...,85.0,86.0,90.0,88.0,93.0,93.0,96.0,93.0,94.0,92.0
975231,Viver i Serrateix,España,80.0,85.0,83.0,80.0,77.0,81.0,73.0,76.0,...,84.0,84.0,87.0,87.0,90.0,90.0,93.0,91.0,93.0,91.0


As we can see, we have the rows for every city repeated three times. We only want the first appearence of each row which is the sum of the other two.

In [35]:
population_clean[population_clean['Poblacion'] == 'Barcelona']

,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
938750,Barcelona,Total,1636193.0,1636732.0,1664182.0,1636762.0,1620343.0,1620809.0,1608746.0,1604555.0,...,1620943.0,1615448.0,1619337.0,1621537.0,1615908.0,1595110.0,1605602.0,1593075.0,1578546.0,1582738.0
938751,Barcelona,España,1158469.0,1178518.0,1202222.0,1215807.0,1227613.0,1238710.0,1247166.0,1251316.0,...,1269957.0,1273594.0,1279809.0,1285885.0,1296748.0,1308454.0,1325249.0,1340386.0,1358272.0,1384815.0
938752,Barcelona,Extranjero,477724.0,458214.0,461960.0,420955.0,392730.0,382099.0,361580.0,353239.0,...,350986.0,341854.0,339528.0,335652.0,319160.0,286656.0,280353.0,252689.0,220274.0,197923.0
951190,Barcelona,Total,778674.0,778744.0,790708.0,775619.0,765962.0,766625.0,761487.0,759520.0,...,769819.0,767915.0,770044.0,771570.0,767697.0,755370.0,761870.0,754463.0,746045.0,748149.0
951191,Barcelona,España,546424.0,555702.0,566671.0,572935.0,578076.0,582992.0,586559.0,587843.0,...,594641.0,595853.0,598232.0,600299.0,605102.0,610761.0,618890.0,625605.0,634356.0,647314.0
951192,Barcelona,Extranjero,232250.0,223042.0,224037.0,202684.0,187886.0,183633.0,174928.0,171677.0,...,175178.0,172062.0,171812.0,171271.0,162595.0,144609.0,142980.0,128858.0,111689.0,100835.0
963630,Barcelona,Total,857519.0,857988.0,873474.0,861143.0,854381.0,854184.0,847259.0,845035.0,...,851124.0,847533.0,849293.0,849967.0,848211.0,839740.0,843732.0,838612.0,832501.0,834589.0
963631,Barcelona,España,612045.0,622816.0,635551.0,642872.0,649537.0,655718.0,660607.0,663473.0,...,675316.0,677741.0,681577.0,685586.0,691646.0,697693.0,706359.0,714781.0,723916.0,737501.0
963632,Barcelona,Extranjero,245474.0,235172.0,237923.0,218271.0,204844.0,198466.0,186652.0,181562.0,...,175808.0,169792.0,167716.0,164381.0,156565.0,142047.0,137373.0,123831.0,108585.0,97088.0


We can drop the repeated rows with `drop_duplicates` method using `subset` and `keep` parameters.

In [36]:
population_clean.drop_duplicates(subset=['Poblacion', 'Origen'], keep='first', inplace=True)

We make a reset of the index after dropping all the unnecessary rows to avoid future index problems.

In [37]:
population_clean.reset_index(inplace=True)
population_clean.drop(columns=['index'],inplace=True)

As we can see the number of rows has dropped from 72936 to 24261 after cleaning the data.

In [38]:
population_clean

,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,2015,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,Alaior,Total,9606.0,9477.0,9467.0,9065.0,9012.0,9097.0,8959.0,8997.0,...,9610.0,9600.0,9399.0,9257.0,9133.0,8972.0,8933.0,8671.0,8308.0,8197.0
1,Alaior,España,7927.0,7884.0,7778.0,7683.0,7633.0,7623.0,7574.0,7643.0,...,7612.0,7610.0,7542.0,7489.0,7443.0,7370.0,7328.0,7249.0,7204.0,7210.0
2,Alaior,Extranjero,1679.0,1593.0,1689.0,1382.0,1379.0,1474.0,1385.0,1354.0,...,1998.0,1990.0,1857.0,1768.0,1690.0,1602.0,1605.0,1422.0,1104.0,987.0
3,Alaró,Total,5811.0,5741.0,5617.0,5572.0,5494.0,5389.0,5282.0,5275.0,...,5508.0,5438.0,5401.0,5327.0,5178.0,5018.0,4869.0,4707.0,4607.0,4540.0
4,Alaró,España,4411.0,4393.0,4344.0,4326.0,4334.0,4330.0,4309.0,4322.0,...,4282.0,4246.0,4230.0,4208.0,4139.0,4113.0,4024.0,3907.0,3885.0,3817.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24256,Vilobí del Penedès,España,1046.0,1047.0,1012.0,1021.0,1016.0,1007.0,1013.0,993.0,...,1007.0,1008.0,1024.0,1024.0,1016.0,996.0,969.0,957.0,926.0,911.0
24257,Vilobí del Penedès,Extranjero,84.0,81.0,90.0,86.0,90.0,87.0,90.0,92.0,...,99.0,95.0,105.0,88.0,93.0,75.0,64.0,56.0,44.0,33.0
24258,Viver i Serrateix,Total,166.0,175.0,172.0,168.0,164.0,173.0,162.0,168.0,...,178.0,178.0,188.0,186.0,193.0,192.0,194.0,193.0,195.0,192.0
24259,Viver i Serrateix,España,164.0,171.0,170.0,166.0,162.0,168.0,158.0,164.0,...,175.0,175.0,182.0,183.0,189.0,186.0,189.0,188.0,193.0,190.0


If we sum the total population by year we get the official population of Spain for every year.

In [39]:
population_total = population_clean[population_clean['Origen'] == 'Total']

In [40]:
population_total[2022].sum()

47249832.0


The final step is to save the filtered and cleaned DataFrame population_clean into a CSV file named 'population_clean.csv'. This file will be used to work in Power BI, enabling analysis and visualization with the processed and appropriately prepared population data.

In [41]:
population_clean.to_csv('population_clean.csv')